## Momentum 

Momentum-based Trading is based on the assumption that Stocks which have performed will in the past, will perform better in the coming future.
 
To define 'past', we take a variable **N**, and say that : 

<centre> Momentum(For A particular stock) = Close Price(Today) - Close Price(N-day ago) </centre>

This gives us our first hyper-parameter (parameters of the model which could be changed in order to improve the model) : **N**

We would also be reshuffling our [Portfolio](https://www.investopedia.com/terms/p/portfolio.asp) at certain intervals of time, which gives us our second hyper-parameter: **T** (The time after which we'll be reshuffling our Portfolio)

Its never suggested to keep all your money invested, you must have some risk-free assets as well, so that even if you lose some of your cash in trading, you could still place better bets and regain that lost cash, Thus, We get our third Hyper-parameter: **R**, The Ratio of Total Balance, which we will using for investing.

You will not be investing in all the 30 Tickers now, Will you? You will choose the top few stocks, which show the highest promise in terms of Momentum, which brings us to another hyper-parameter: **M**, The Number of Top few stocks (based on Momentum), which you'll keep in your Portfolio.

Finally, There's some brokerage fee which you need to pay in order to place orders on the stock market, typically its less than 0.05% of the total amount : **F**


In [3]:
#Importing Required Libraries
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
import seaborn as sns

#Declaring the Hyperparameters

N = 50
T = 7
R = 0.8
M = 5
F = 0.0005   # 0.5% Brokerage fee

The Second step would be to define a function which reads the Prices of various Stocks into memory.

In the file DATA.csv , which we had uploaded in our repository, we have prices of 30 firms enlisted in S & P 500 Index (Apple, IBM, Cisco, Walmart and the like!) from 2nd January 2009 to 17th August 2020.

For our purposes, We'll only be requiring certain columns. On an honest note, Just getting the Columns on Ticker, Date and Adjusted Closing Price would do the job, but if you want, you may take Opening Price as well.

Read up about the [pandas.read_csv](https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html) function from here, and figure out how you'll use it to do the job (You don't need all the columns!) 

In [47]:
def GetData(NameOfFile):
  #START CODE HERE
    df=pd.read_csv(NameOfFile)
    d=df[['datadate','tic','adjcp']]
    
    return d # pd.DataFrame Object

To aid Data-Manipulation, it would be beneficial, if we split the DataFrame into many small parts each corresponding to the data corresponding to the 30 Tickers on a particular date. These small parts could then be stored in a list.

We would also be needing to remember which date is at what index, so that we can use that later. 

In [53]:
def PartitionData(Data):
  
    DateToIndex = {}

    dataByDate=[]
    
    grouped=Data.groupby('datadate')
    
    for i,(date,group) in enumerate(grouped):
        DateToIndex[date]=i
        dataByDate.append(group[['tic','adjcp']])

    return dataByDate, DateToIndex   # List containing of the Data Partitioned according to Date, and the Dictionary mapping Dates to their index in the list 

Now, We need a function to calculate the Momentum value for all of our 30 Tickers.
To do this, We need to have a few things in mind:


1.   We need to start at Nth day in our list, as only then we'll be able to calculate the Momentum (This will be taken care of by later parts of the Program, when we actually run the Model)

2.   The Stock Market isn't open on all days, so we often won't be able to go N days behind, and will have to take the closest value instead(We can't just go N entries behind in the List we created and expect that to work, Why?) In order to work with dates, you should get to know more about the datetime library of Python from [here](https://thispointer.com/python-how-to-convert-datetime-object-to-string-using-datetime-strftime/) (Especially the datetime.strftime() function) and about the [datetime.timedelta()](https://www.studytonight.com/python-howtos/how-to-add-days-to-date-in-python) function.

Also, as you may have figured it out yourself, while DataFrames are great for Data Handling and small statistical calculations, They aren't so for big calculations as the Numpy Library has both a richer variety of functions for such manipulations and is also more efficient!

After we calculate the Momentum for all our Tickers, it would be a great thing to do, if we could divide their prices by their mean(in the N day interval, as we need to see which stock outperforms others and in order to do this, it won't be fair if we take the absolute growth in numbers!(Why?)



In [93]:
def date_to_str(date):
    date=str(date).split('-')
    date=date[0]+date[1]+date[2]
    return date

def GetMomentumBasedPriority(PartitionedDataFrameList, DateToIndex ,today):
    # PartitionedDataFrameList : Pandas DataFrame, The Output of your last function
    # DateToIndex : Dictionary mapping dates to their index in the PartitionedDataFrameList
    # today :  Today's date (string) In Format: YYYYMMDD


    #NdaysAgo is a datatime.date() object contining the required data, you need to convert it to a string and then check if its
    #actually there in the Data you have or will you have to get going using some other nearest date

    NdaysAgo = datetime.date(int(today[0:4]),int(today[4:6]),int(today[6:])) + datetime.timedelta(days = -N)

    #START CODE HERE!
    l=date_to_str(NdaysAgo)
    
    if l in DateToIndex.keys():
        index=DateToIndex[l]
    else:
        n1=NdaysAgo
        n2=NdaysAgo
        i1=n1
        i2=n2
        
        while True:
            n1=n1+datetime.timedelta(days=-1)
            if date_to_str(n1) in DateToIndex.keys():
                i1=n1
                break
                
        while True:
            n2=n2+datetime.timedelta(days=-1)
            if date_to_str(n2) in DateToIndex.keys():
                i2=n2
                break
                
        if int(str(n2-NdaysAgo)[0])>int(str(NdaysAgo-n1)[0]):
            index=date_to_str(i1)
        else:
            index=date_to_str(i2)
        
        
    array_today=PartitionedDataFrameList[DateToIndex[today]]['adjcp'].to_numpy()
    array_N_day_ago=PartitionedDataFrameList[DateToIndex[index]]['adjcp'].to_numpy()
    
    momentum_array=array_today-array_N_day_ago
    momentum_array=momentum_array/(np.mean(momentum_array,axis=0))

    return momentum_array   #Numpy Array containing the Momentum divided by mean(in the N-day period considered) of all the 30 tickers, in the required order.



Even after you have got your Momentum-based priorities, and have decided which stocks to buy and what will be the weight of each, you still need to figure out how much of each will you buy. To do this, first you'll sell all your pre-owned stocks which will increase your cash in hand, then you'll know the stocks to buy and their relative weights (given by their Momentum/mean) and you need a function which tells you how many stocks to buy for each ticker!

In [94]:
def GetBalanced(prices, weights,balance):
    # prices : Numpy array containing Prices of all the 30 Stocks
    # weights : Multi-hot Numpy Array : The Elements corresponding to stocks which are to be bought(Top M Stocks with positive Momentum Indicator) are set to their priority, All other elements are set to zero.
    # Returns Numpy array containing the number of shares to buy for each stock!
    
    amount_Invested=(R*balance)/(1+F)
    weights=weights/(np.sum(weights,axis=0))
    number_of_stocks=(amount_Invested*weights)//prices
    
    return number_of_stocks

Now, We need something to simulate our [Portfolio](https://www.investopedia.com/terms/p/portfolio.asp). In order to do that we need a class, which has certain  basic features and functionalities.

Features : 


1.   Your Initial Balance
2.   Your Current Balance
3.   A list(/any other container) storing the number of stocks of each ticker currently in possession. (Numpy Array prefered)
4.   Most recent prices of all the stocks (As a Numpy array)

Functionalities: 



1.   Calculating current Net Worth (Balance+Total Evaluation of all Stocks owned!) 
2.   Buying a Particular Stock (Keep the Transaction fee in mind!)
3.   Selling a particular Stock whole (Keep the Transaction Fee in mind!)
4.   Rebalance Portfolio  (Takes Numpy array as input)
5.   Function to change the value of most recent prices stored (Takes Numpy array as input)





In [95]:
class PortFolio:
    def __init__(self,initial_balance,curr_balance,possession,recent_prices):
        self.init_balance=initial_balance
        self.curr_balance=curr_balance
        self.possession=possession
        self.recent_prices=recent_prices


    def SellStock(self,index):
        #index : The index of the Stock to sell (0-29)      
        stock_price=self.recent_prices[index]*self.possession[index]
        self.curr_balance+=stock_price*(1-F)
        self.possession[index]=0
    

    def BuyStock(self,index, number):
        #index : The index of the Stock to buy (0-29) 
        #number : Number of shares to buy (float)
        stock_price=number*self.recent_prices[index]
        self.curr_balance-=stock_price*(1+F)
        self.possession+=number
        

    def CalculateNetWorth(self):
        #Return Net Worth (All Shares' costs+ Balance)
        return self.curr_balance+np.sum(np.multiply(self.possession,self.recent_prices))


    def ChangePricesTo(self,newPriceVector):
        # newPriceVector : Numpy array containing the prices of all the stocks for the current day
        self.recent_prices=newPriceVector


    def RebalancePortFolio(self,newWeights):  
        # newWeights : Numpy array containing Momentum/Mean for all stocks in the N-day period
        # First sell all your pre-owned Stock (make sure to take transaction fee into account!)
        # This would change your total balance
        # Then take the top M(/might be less than M, if >(30-M) had negative Momentum) and send them to the GetBalanced() Function
        # Then take that output and pass them to the BuyStocks function!
        self.curr_balance+=np.sum(np.multiply(self.possession,self.recent_prices))
        
        weight_dic={}
        for i in range(len(newWeights)):
            weight_dic[i]=newWeights[i]
            
        sorted_weights=np.sort(newWeights)
        
        weights=np.zeros(30)
        for i in range(M):
            if sorted_weights[i]>0:
                for j in weight_dic.keys():
                    if newWeights[j]==sorted_weights[i]:
                        weights[j]=newWeights[j]
        
        # weights is the updates array
        self.possession=GetBalanced(self.recent_prices,weights,self.curr_balance)
        self.curr_balance=(1-R)*self.curr_balance
        
        

With that the difficult part is over!

Now, all you need to work on is a main loop, which calls all these functions

In [ ]:
myPortfolio = PortFolio(100000,100000,0,0)
NetWorthAfterEachTrade = []


#First Get the Data
Data = GetData('DATA.csv')
PartitionedData, DateToIndex= PartitionData(Data)

#
networth=[]
indexToDate=dict((index,str(date)) for date, index in DateToIndex.items())

reshuffling_period=T

#Start processing from the (N+1)th Day(among the ones recorded in the Data)
for i in range(N+1,len(DateToIndex)):
    # Change the Prices to the ith Term
    # Get NetWorth and store in list
    # Check if you need to rebalance Portfolio's Today
    # If so, do it by Calling first the GetMomentumBasedPriority function and then passing it to the rebalance function
    df=PartitionedData[i]
    prices=df['adjcp'].to_numpy()
    
    myPortfolio.ChangePricesTo(prices)
    NetWorthAfterEachTrade.append(myPortfolio.CalculateNetWorth())
    
    if reshuffling_period==0:      
        momentum_array=GetMomentumBasedPriority(PartitionedData, DateToIndex, indexToDate[i])
        myPortfolio.RebalancePortfolio(momentum_array)
        reshuffling_period=T
    reshuffling_period-=1
  

##Moment of Truth

Time to check, if your Program actually works!

Plot the data you collected in various ways and see if what you did worked!

Feel free to use whichever one of Matplotlib or Seaborn you want to.

You should try changing the hyper-parameters to increase(/decrease) your performance!


In [ ]:
from matplotlib import pyplot as plt
def VizualizeData():









  

You may use this cell to write about what results you got!